In [ ]:
from skimage import io, measure
import matplotlib.pyplot as plt
import os
import numpy as np 
import napari
import h5py
from tqdm import tqdm
import statistics
import scipy.ndimage as ndi
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
from pathlib import Path

# Read data

In [22]:
im_folder = "/Users/thomasathey/Documents/shavit-lab/jellyfish/CT_scan_cand13/tifs"
im_folder = "/Users/thomasathey/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/jellyfish-imaging/25_01_07tommy_ct_2025-01-06_170445/tiffs"

res = 1.382273

res_3d = (res, res, res)

## Collect 2d images from im_folder

In [ ]:
# identify all tifs in im_folder and sort them by their name
im_files = os.listdir(im_folder)
im_files = [f for f in im_files if f.endswith('.tiff')]
im_files.sort()

# read all tifs and stack them into a 3D array
im_stack = []
for f in tqdm(im_files):
    im = io.imread(os.path.join(im_folder, f))
    im_stack.append(im)
im_stack = np.array(im_stack)

image = im_stack

im_stack.shape

## Read a 3d image

In [ ]:
image = io.imread("/Users/thomasathey/Documents/shavit-lab/jellyfish/CT_scan_cand13/3d_processed_files/im_full.tif")

print(f"Image shape: {image.shape}")

In [ ]:
seg_obj = io.imread("/Users/thomasathey/Documents/shavit-lab/jellyfish/CT_scan_cand13/3d_processed_files/obj_full.tif")
seg_subum = io.imread("/Users/thomasathey/Documents/shavit-lab/jellyfish/CT_scan_cand13/3d_processed_files/subum_full.tif")

In [ ]:
folder = Path("/Users/thomasathey/Documents/shavit-lab/jellyfish/CT_scan_cand13/3d_processed_files")

image_crop = io.imread(folder / "im_hemi.tif")
seg_subum_crop = io.imread(folder / "subum_hemi.tif")
seg_obj_crop = io.imread(folder / "obj_hemi.tif")

## Read new 3d image

In [15]:
image = io.imread("/Users/thomasathey/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/jellyfish-imaging/25_01_07tommy_ct_2025-01-06_170445/derivatives/im_full.tif")

print(f"Image shape: {image.shape}")

Image shape: (993, 1013, 990)


## View data in napari

### Image only in unit resolution

In [ ]:
# io.imsave("/Users/thomasathey/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/jellyfish-imaging/25_01_07tommy_ct_2025-01-06_170445/derivatives/im_full.tif", image)

In [16]:
image_crop_rough = np.copy(image)#[:940,:,:] #image[:940,200:,:]
image_crop_rough[:300,:,:] = 0

In [23]:
# show im_stack in a napari viewer
viewer = napari.Viewer()

viewer.add_image(image_crop_rough, scale=res_3d)
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

### True resolution

In [ ]:
# show im_stack in a napari viewer
viewer = napari.Viewer()

viewer.add_image(image, scale=res_3d)
viewer.add_labels(seg_obj, scale=res_3d)
viewer.add_labels(seg_subum, scale=res_3d)
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

In [ ]:
# show im_stack in a napari viewer
viewer = napari.Viewer()

viewer.add_image(image_crop, scale=res_3d)
viewer.add_labels(seg_obj_crop, scale=res_3d)
viewer.add_labels(seg_subum_crop, scale=res_3d)
viewer.scale_bar.unit = 'um'
viewer.scale_bar.visible = True

In [ ]:
p1 = (973,1026)
p2 = (1112, 1140)

np.linalg.norm(np.subtract(p2,p1)*res)

# Gonad coords

In [17]:
# zeiss viewer
a = [496, 712, 788]
b = [356,410, 721]
c = [674, 686, 456]
d = [456, 388, 373]

mouth = [563, 564, 467]

# reverse into napari coordinates
a.reverse()
b.reverse()
c.reverse()
d.reverse()

points = np.array([a,b,c,d])

## Normal vector

In [19]:
v1 = np.subtract(b,c)
v2 = np.subtract(b,mouth)

v1 = v1.astype(float)
v2 = v2.astype(float)

# v1 /= np.linalg.norm(v1)
# v2 /= np.linalg.norm(v2)

normal = np.cross(v1,v2)
normal /= np.linalg.norm(normal)

normal2 = (normal[-1], normal[1], normal[2])

In [20]:
# show im_stack in a napari viewer
viewer = napari.Viewer()

viewer.add_image(image)
viewer.add_image(image_crop_rough)
viewer.add_points([a], size=10, face_color='red', border_color='white')
viewer.add_points([b,c], size=10, face_color='orange', border_color='white')
viewer.add_points([d], size=10, face_color='green', border_color='white')
viewer.add_points([mouth], size=10, face_color='blue', border_color='white')



plane_parameters = {
    'position': (0, 0, 0),
    'normal': normal
    }
plane_layer = viewer.add_image(
    image,
    name='plane',
    rendering='average',
    depiction='plane',
    blending='additive',
    colormap='cyan',
    plane=plane_parameters
    )
